In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-27 23:41:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.78MB/s    in 0.2s    

2021-05-27 23:41:39 (4.78 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Data into Spark DataFrame and Create DataFrames to match tables**

In [6]:
# Create product dataframe
from pyspark import SparkFiles
url = "https://wrennk-ice-cream.s3.amazonaws.com/products.csv"
spark.sparkContext.addFile(url)
products_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("products.csv"), sep=",", header=True, inferSchema=True)
products_df.show(5)

+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
|brand| key|               name|             subhead|         description|rating|rating_count|         ingredients|
+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
|   bj|0_bj|Salted Caramel Core|Sweet Cream Ice C...|Find your way to ...|   3.7|         208|CREAM, SKIM MILK,...|
|   bj|1_bj|Netflix & Chilll'd™|Peanut Butter Ice...|There’s something...|   4.0|         127|CREAM, SKIM MILK,...|
|   bj|2_bj|       Chip Happens|A Cold Mess of Ch...|Sometimes “chip” ...|   4.7|         130|CREAM, LIQUID SUG...|
|   bj|3_bj|            Cannoli|Mascarpone Ice Cr...|As a Limited Batc...|   3.6|          70|CREAM, SKIM MILK,...|
|   bj|4_bj|     Gimme S’more!™|Toasted Marshmall...|It’s a gimme: the...|   4.5|         281|CREAM, SKIM MILK,...|
+-----+----+-------------------+--------------------+-------------------

In [7]:
# Create review dataframe
url = "https://wrennk-ice-cream.s3.amazonaws.com/reviews.csv"
spark.sparkContext.addFile(url)
reviews_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("reviews.csv"), sep=",", header=True, inferSchema=True)
reviews_df.show(5)

+-------+------------+--------------+----------+--------------------+--------------------+-----------+----------+--------------------+-----+-----------+-------+-----+
|  brand|         key|        author|      date|               stars|               title|helpful_yes|helpful_no|                text|taste|ingredients|texture|likes|
+-------+------------+--------------+----------+--------------------+--------------------+-----------+----------+--------------------+-----+-----------+-------+-----+
|     bj|        0_bj|Ilovebennjerry|2017-04-15|                   3|Not enough brownies!|       10.0|       3.0|Super good, don't...| null|       null|   null| null|
|Overall| good flavor|       texture|      idea| and brownies. No...|                null|       null|      null|                null| null|       null|   null| null|
|     bj|        0_bj| Sweettooth909|2020-01-05|                   5|I’m OBSESSED with...|        3.0|       0.0|I decided to try ...| null|       null|   null| null

In [14]:
# Create clean_review dataframe
url = "https://wrennk-ice-cream.s3.amazonaws.com/clean_reviews.csv"
spark.sparkContext.addFile(url)
clean_reviews_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("clean_reviews.csv"), sep=",", header=True, inferSchema=True)
clean_reviews_df.show(5)

+----+-----+-----------+----------+--------------------+
| key|stars|helpful_yes|helpful_no|                text|
+----+-----+-----------+----------+--------------------+
|0_bj|    3|       10.0|       3.0|Super good, dont ...|
|0_bj|    5|        3.0|       0.0|I decided to try ...|
|0_bj|    3|        5.0|       2.0|My caramel core b...|
|0_bj|    5|       24.0|       1.0|Why are people co...|
|0_bj|    2|        3.0|       1.0|I bought this las...|
+----+-----+-----------+----------+--------------------+
only showing top 5 rows



In [10]:
# Create helpful_clean_review dataframe
url = "https://wrennk-ice-cream.s3.amazonaws.com/helpful_clean_reviews.csv"
spark.sparkContext.addFile(url)
helpful_clean_reviews_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("helpful_clean_reviews.csv"), sep=",", header=True, inferSchema=True)
helpful_clean_reviews_df.show(5)

+----+-----+-----------+----------+--------------------+
| key|stars|helpful_yes|helpful_no|                text|
+----+-----+-----------+----------+--------------------+
|0_bj|    3|       10.0|       3.0|Super good, dont ...|
|0_bj|    5|        3.0|       0.0|I decided to try ...|
|0_bj|    3|        5.0|       2.0|My caramel core b...|
|0_bj|    5|       24.0|       1.0|Why are people co...|
|0_bj|    2|        3.0|       1.0|I bought this las...|
+----+-----+-----------+----------+--------------------+
only showing top 5 rows



In [11]:
# Create high_rating dataframe
url = "https://wrennk-ice-cream.s3.amazonaws.com/high_rating.csv"
spark.sparkContext.addFile(url)
high_rating_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("high_rating.csv"), sep=",", header=True, inferSchema=True)
high_rating_df.show(5)

+----+--------------------+--------------------+------+------------+
| key|                name|         description|rating|rating_count|
+----+--------------------+--------------------+------+------------+
|1_bj| Netflix & Chilll'd™|There’s something...|   4.0|         127|
|2_bj|        Chip Happens|Sometimes “chip” ...|   4.7|         130|
|4_bj|      Gimme S’more!™|It’s a gimme: the...|   4.5|         281|
|5_bj|Peanut Butter Hal...|If you were more ...|   4.9|          14|
|6_bj|Berry Sweet Masca...|From the sweet bl...|   4.6|          10|
+----+--------------------+--------------------+------+------------+
only showing top 5 rows



In [17]:
# Create combined dataframe
url = "https://wrennk-ice-cream.s3.amazonaws.com/combined.csv"
spark.sparkContext.addFile(url)
combined_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("combined.csv"), sep=",", header=True, inferSchema=True)
combined_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+----------+----+
|                 key|                name|         description|              rating|        rating_count|stars|         helpful_yes|helpful_no|text|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+----------+----+
|           0_breyers|     Natural Vanilla|Our Original Vani...|                null|                null| null|                null|      null|null|
|When William Brey...| he based his rec...|               sugar| and milk? To mak...| the milk and cre...| null|                null|      null|null|
|Do you see those ...| so you can enjoy...| knowing that you...|                null|                null| null|                null|      null|null|
|Try Breyers® Natu...|                null|                null|                null|               

**Connect to the AWS RDS instance and write each DataFrame to its Table**